### Created by: Rodrigo Didier, 01/31/21.

# Import Libs

In [25]:
import sys
import os
import re
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split

import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

from nltk.corpus import stopwords
from string import punctuation

from nltk.probability import FreqDist

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# DATA SCIENCE PIPLINE --------------------------------------------------------------------------

# 0. Feature Engineering

# 1. Data Extraction

In [6]:
class dataExtractor:

    def __init__(self):
        self.data_path = os.getenv('DATASET_PATH')
        self.data = self.extractData()

    def extractData(self):
        '''
        Loads a dataset with product data from a specified path.
        '''

        return pd.read_csv(self.data_path)

#  2. Data Formatting

In [48]:
#X.head()
(X, y), (X_test, y_test), (X_val, y_val) = dataFormatter().normalizeFullText()

In [53]:
X_val.full_text[1343]

'ladybug lembrancinhas tag lembrancinha ladybug festa ladybug sofia isadora ladybug aniversario mica'

In [27]:
test = X.full_text[12] + ', pé-de-moleque. Vai!'

In [31]:
test

'cha de panela Mini Tabua de Carne Lembrancinhas De Casamento/chá De Panela casamento cha panela, pé-de-moleque. Vai!'

In [30]:
normalize_answer(test)

'cha panela mini tabua carne lembrancinhas casamento chá panela casamento cha panela pé moleque vai'

In [ ]:
def 

In [128]:
def remove_punctuation(value):
    result = ""
    for c in value:
        # If char is not punctuation, add it to the result.
        if c not in punctuation + '/':
            result += c
    return result.lower()


In [50]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [125]:
text = X.full_text[12]
text = text + ' pé-de-moleque, asd , ,'

In [44]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""


    def compostWords(text):
        text = re.sub('[/]', ' ', str(text))
        text = re.sub('[-]', ' ', str(text))
        return text

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()
    
    def removeStopWords(text):
        from nltk.corpus import stopwords
        stopwords = set(stopwords.words('portuguese') + list(punctuation) + list('/'))
        palavras = word_tokenize(text)
        palavras_sem_stopwords = [palavra for palavra in palavras if palavra not in stopwords]
        return " ".join(palavras_sem_stopwords)

    return removeStopWords(white_space_fix(remove_punc(lower(compostWords(s)))))

In [37]:
normalize_answer(test)

'cha panela mini tabua carne lembrancinhas casamento chá panela casamento cha panela pé moleque vai'

In [40]:
list(X['full_text'])

['tags Tags Etiqueta para Artesanato etiquetas papel tags personalizadas tags',
 'lembrancinhas para o dia dos pais evangelicos Placa Homenagem Personalizada com estojo dia pais placas homenagem personalizadas',
 'quadro abstrato Quadros Abstratos Britto QB083 160x80cm abstratos 160x80cm',
 'quadro decorativo Quadros Decorativos - Gato niver 50 anos quadrinhos parede',
 'copo degrade Copo Long drink Degradê Formatura copos canecas acrilico festa decoracao casamento formatura canecas acrilico',
 'patrulha canina 50 Apliques Patrulha Canina patrulha canina papelaria festa 4o aniversario lucas j tema contrucao niver',
 'capas para portifolio eva Pasta portfólio de Pinos-c/ botão e fita lembrancas mimos ficharios portfolios pinos produtos escritorio',
 'resina Decoração de resina acessorios trabalhos enxoval bebe resinas',
 'lembrancinhas para padrinhos Lembrancinha Padrinhos casamento lembrancas casamento lembranca padrinhos juliana patrick cha lingerie/bar manual kit padrinhos casamento 

In [45]:
list(map(normalize_answer,list(X['full_text'])))

['tags tags etiqueta artesanato etiquetas papel tags personalizadas tags',
 'lembrancinhas dia pais evangelicos placa homenagem personalizada estojo dia pais placas homenagem personalizadas',
 'quadro abstrato quadros abstratos britto qb083 160x80cm abstratos 160x80cm',
 'quadro decorativo quadros decorativos gato niver 50 anos quadrinhos parede',
 'copo degrade copo long drink degradê formatura copos canecas acrilico festa decoracao casamento formatura canecas acrilico',
 'patrulha canina 50 apliques patrulha canina patrulha canina papelaria festa 4o aniversario lucas j tema contrucao niver',
 'capas portifolio eva pasta portfólio pinos c botão fita lembrancas mimos ficharios portfolios pinos produtos escritorio',
 'resina decoração resina acessorios trabalhos enxoval bebe resinas',
 'lembrancinhas padrinhos lembrancinha padrinhos casamento lembrancas casamento lembranca padrinhos juliana patrick cha lingerie bar manual kit padrinhos casamento caixinhas convite',
 'sacola lembrancinha

In [36]:
X.full_text.apply(lambda x: normalize_answer(x))

TypeError: expected string or bytes-like object

In [142]:
text = text.replace("!@#$%^&*()[]{};:,./<>?\|`~-=_+", " ")
text = re.sub('[/-]', ' ', text)
#palavras_sem_especial = [letra for letra in letras if letras not in "!@#$%^&*()[]{};:,./<>?\|`~-=_+"]
text = text.lower()

In [134]:
palavras_sem_especial = [x for x in text.split() if text.split() not in "!@#$%^&*()[]{};:,./<>?\|`~-=_+"]

TypeError: 'in <string>' requires string as left operand, not list

In [130]:
text.split()

['cha',
 'de',
 'panela',
 'mini',
 'tabua',
 'de',
 'carne',
 'lembrancinhas',
 'de',
 'casamento',
 'chá',
 'de',
 'panela',
 'casamento',
 'cha',
 'panela',
 'pé',
 'de',
 'moleque,',
 'asd',
 ',',
 ',']

In [110]:
text.replace("!@#$%^&*()[]{};:,./<>?\|`~-=_+", " ")

'cha de panela Mini Tabua de Carne Lembrancinhas De Casamento/chá De Panela casamento cha panela'

In [80]:
re.findall("/", text)

['/']

In [73]:
remove_punctuation(X.full_text[12])

'cha de panela mini tabua de carne lembrancinhas de casamentochá de panela casamento cha panela'

In [150]:
sentencas

['cha de panela mini tabua de carne lembrancinhas de casamento chá de panela casamento cha panela pé de moleque, asd , ,']

In [163]:
def clean(sentence):
    #sentencas = sent_tokenize(X.full_text[12])
    sentence = sentence.replace("!@#$%^&*()[]{};:,./<>?\|`~-=_+", " ")
    sentence = re.sub('[/]', ' ', sentence)
    sentence = re.sub('[-]', ' ', sentence)
    sentence = re.sub('[,]', ' ', sentence)
    sentence = re.sub('[:]', ' ', sentence)
    sentence = re.sub('[;]', ' ', sentence)
    sentence = " ".join(sentence.split())
    palavras = word_tokenize(sentence.lower())
    sentence = ' '.join(palavras)
    stopwords = set(stopwords.words('portuguese') + list(punctuation) + list('/'))
    palavras_sem_stopwords = [palavra for palavra in palavras if palavra not in stopwords]
    return " ".join(palavras_sem_stopwords)
    #frequencia = FreqDist(palavras_sem_stopwords)

In [ ]:
stop_words = nltk.corpus.stopwords.words("portuguese") +\
             nltk.corpus.stopwords.words("english")

for i in text.splitlines():
    clean_text = gensim.utils.simple_preprocess(i)
    clean_text = [i for i in clean_text if i not in stop_words]
    sentences.append(clean_text)

In [164]:
clean(X.full_text[12])

UnboundLocalError: local variable 'stopwords' referenced before assignment

In [155]:
' '.join(palavras)

'cha de panela mini tabua de carne lembrancinhas de casamento/chá de panela casamento cha panela'

In [156]:
text

'cha de panela mini tabua de carne lembrancinhas de casamento chá de panela casamento cha panela'

In [144]:
sentencas = sent_tokenize(text)
palavras = word_tokenize(text)

stopwords = set(stopwords.words('portuguese') + list(punctuation) + list('/'))
palavras_sem_stopwords = [palavra for palavra in palavras if palavra not in stopwords]
frequencia = FreqDist(palavras_sem_stopwords)

In [145]:
palavras

['cha',
 'de',
 'panela',
 'mini',
 'tabua',
 'de',
 'carne',
 'lembrancinhas',
 'de',
 'casamento',
 'chá',
 'de',
 'panela',
 'casamento',
 'cha',
 'panela',
 'pé',
 'de',
 'moleque',
 ',',
 'asd',
 ',',
 ',']

In [68]:
frequencia

FreqDist({'panela': 3, 'cha': 2, 'mini': 1, 'tabua': 1, 'carne': 1, 'lembrancinhas': 1, 'casamento/chá': 1, 'casamento': 1})

In [47]:
class dataFormatter(dataExtractor):
    def __init__(self):
        self.extracted_data = dataExtractor().data
        #self.train_data = self.formatData()[0]
        #self.validation_data = self.formatData()[1]
        
    def formatData(self):
        '''
        Processes the dataset to use it for training and validation.
        '''
        
        # add methods to formating the data to the  modeler.
    
    def categoryToDummy(self):
        # get dummy
        dummy = pd.get_dummies(self.extracted_data['category'])

        # merge df
        df_dummy = pd.merge(self.extracted_data, dummy, left_index=True, right_index=True)

        # delet no dummy cols
        del df_dummy['category']

        return df_dummy
    
    def splitTrainTestValidate(self):
        '''
        Split the dataset into train, validation and test
        '''
        
        # categorical variables list to split
        y_cols = list(self.categoryToDummy().columns[-6:])
        X_cols = list(self.categoryToDummy().columns[:-6])
        
        # full data categorized
        y_data =  self.categoryToDummy()[y_cols] #.values
        X_data =  self.categoryToDummy()[X_cols] #.values
                
        X, X_test, y, y_test = train_test_split(X_data, y_data, test_size=0.2, train_size=0.8)
        X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.25,train_size =0.75)
        
        return (X, y), (X_test, y_test), (X_val, y_val)
    
    def joinStringColumns(self):
        
        (X, y), (X_test, y_test), (X_val, y_val) = self.splitTrainTestValidate()
        f_join_strings = lambda row: row['query'] + ' ' + row['title'] + ' ' + row['concatenated_tags']
        
        X = X.assign(full_text= f_join_strings)
        X_test = X_test.assign(full_text= f_join_strings)
        X_val = X_val.assign(full_text= f_join_strings)
        
        return (X, y), (X_test, y_test), (X_val, y_val)
    
 
    def normalizeFullText(self):
        """Lower text and remove punctuation, articles and extra whitespace."""
    
        
        def normalize_text(s):
            """Lower text and remove punctuation, articles and extra whitespace."""        

            def compost_words(text):
                text = re.sub('[/]', ' ', str(text))
                text = re.sub('[-]', ' ', str(text))
                return text

            def white_space_fix(text):
                return " ".join(text.split())

            def remove_punc(text):
                exclude = set(punctuation)
                return "".join(ch for ch in text if ch not in exclude)

            def lower(text):
                return text.lower()
    
            def remove_stop_words(text):
                from nltk.corpus import stopwords
                stopwords = set(stopwords.words('portuguese') + list(punctuation) + list('/'))
                palavras = word_tokenize(text)
                palavras_sem_stopwords = [palavra for palavra in palavras if palavra not in stopwords]
                return " ".join(palavras_sem_stopwords)
        
            return remove_stop_words(white_space_fix(remove_punc(lower(compost_words(s)))))
    
        (X, y), (X_test, y_test), (X_val, y_val) = self.joinStringColumns()
        X['full_text'] = list(map(normalize_text,list(X['full_text'])))
        X_test['full_text'] = list(map(normalize_text,list(X_test['full_text'])))
        X_val['full_text'] = list(map(normalize_text,list(X_val['full_text'])))
        
        
        return (X, y), (X_test, y_test), (X_val, y_val)
    

import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

texto = 'João pé de feijão, foi correr e sentiu. Porém, seguiu! esse cara é fod* texto'
sentencas = sent_tokenize(texto)
palavras = word_tokenize(texto.lower())

from nltk.corpus import stopwords
from string import punctuation

stopwords = set(stopwords.words('portuguese') + list(punctuation))
palavras_sem_stopwords = [palavra for palavra in palavras if palavra not in stopwords]

from nltk.probability import FreqDist
frequencia = FreqDist(palavras_sem_stopwords)

#from collections import defaultdict
#sentencas_importantes = defaultdict(int)

#for i, sentenca in enumerate(sentencas):
#    for palavra in word_tokenize(sentenca.lower()):
#        if palavra in frequencia:
#            sentencas_importantes[i] += frequencia[palavra]

In [12]:
#frequencia

import re
import numpy as n
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')

# 3. Data Modeling

In [8]:
class dataModeler(dataFormatter):
    def __init__(self):
        self.model = dataModeler.getModel()
        self.train_data = dataFormatter.train_data
        
    def getModel():
        '''
        Specifies a model to handle the categorization problem.
        # criar etapa de modelagem -métodos se for o caso
        # dar split no train e treinar
        
        
        '''

        model = None
        return model
        
    def writeModel(self):
        '''
        Exports a candidate model to a specified path available
        in the environment variable MODEL_PATH.
        '''

        # criar try: tenta salvar, se nao dá erro.
    
        return pickle.dump(self.model, open(os.getenv('MODEL_PATH'),'wb'))


# 4. Model Validation

In [9]:
class modelValidator(dataModeler, dataFormatter):
    def __init__(self):
        self.model = self.readModel()
        self.validation_data = dataFormatter.validation_data
    
    def readModel(self):
        '''
        get the model selected to handle the categorization problem.
        '''
        
        # criar um try, se nao der entao dar um saveModel
        
        return pickle.load(open(os.getenv('MODEL_PATH'),'rb'))
        
    def validateModel(self):
        '''
        Generates metrics about the model accuracy (precision, recall, F1, etc.)
        for each category and exports them to a specified path available in the 
        environment variable METRICS_PATH.
        '''
        # try k-fold and variants.
        
        # should save metrics in METRICS_PATH
        f = open(os.getenv('METRICS_PATH'), "w")
        f.write("F1:95.0, Precision:87.5")
        f.close()
        pass

# 5. Model Prediction